# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc4b4af34f0>
├── 'a' --> tensor([[ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160]])
└── 'x' --> <FastTreeValue 0x7fc4b8bc86a0>
    └── 'c' --> tensor([[-1.1935, -0.9629, -0.5751,  1.0858],
                        [ 1.5296,  0.4821,  0.2280,  0.8139],
                        [ 0.1560, -0.4275, -0.7215, -0.3529]])

In [4]:
t.a

tensor([[ 0.0973,  0.9077,  0.9042],
        [-0.1918,  0.9390,  0.5160]])

In [5]:
%timeit t.a

61.5 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc4b4af34f0>
├── 'a' --> tensor([[-1.1666, -0.6799,  0.8270],
│                   [-1.0345, -0.8299, -1.1693]])
└── 'x' --> <FastTreeValue 0x7fc4b8bc86a0>
    └── 'c' --> tensor([[-1.1935, -0.9629, -0.5751,  1.0858],
                        [ 1.5296,  0.4821,  0.2280,  0.8139],
                        [ 0.1560, -0.4275, -0.7215, -0.3529]])

In [7]:
%timeit t.a = new_value

75.6 ns ± 1.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160]]),
    x: Batch(
           c: tensor([[-1.1935, -0.9629, -0.5751,  1.0858],
                      [ 1.5296,  0.4821,  0.2280,  0.8139],
                      [ 0.1560, -0.4275, -0.7215, -0.3529]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0973,  0.9077,  0.9042],
        [-0.1918,  0.9390,  0.5160]])

In [11]:
%timeit b.a

53.5 ns ± 0.0233 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3574, -1.8670,  1.3019],
               [-0.6099, -0.4065, -0.4254]]),
    x: Batch(
           c: tensor([[-1.1935, -0.9629, -0.5751,  1.0858],
                      [ 1.5296,  0.4821,  0.2280,  0.8139],
                      [ 0.1560, -0.4275, -0.7215, -0.3529]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.0861 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 0.225 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 28.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 266 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 668 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc40e7b8a60>
├── 'a' --> tensor([[[ 0.0973,  0.9077,  0.9042],
│                    [-0.1918,  0.9390,  0.5160]],
│           
│                   [[ 0.0973,  0.9077,  0.9042],
│                    [-0.1918,  0.9390,  0.5160]],
│           
│                   [[ 0.0973,  0.9077,  0.9042],
│                    [-0.1918,  0.9390,  0.5160]],
│           
│                   [[ 0.0973,  0.9077,  0.9042],
│                    [-0.1918,  0.9390,  0.5160]],
│           
│                   [[ 0.0973,  0.9077,  0.9042],
│                    [-0.1918,  0.9390,  0.5160]],
│           
│                   [[ 0.0973,  0.9077,  0.9042],
│                    [-0.1918,  0.9390,  0.5160]],
│           
│                   [[ 0.0973,  0.9077,  0.9042],
│                    [-0.1918,  0.9390,  0.5160]],
│           
│                   [[ 0.0973,  0.9077,  0.9042],
│                    [-0.1918,  0.9390,  0.5160]]])
└── 'x' --> <FastTreeValue 0x7fc40e7b8850>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc4b4af34c0>
├── 'a' --> tensor([[ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160],
│                   [ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160],
│                   [ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160],
│                   [ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160],
│                   [ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160],
│                   [ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160],
│                   [ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160],
│                   [ 0.0973,  0.9077,  0.9042],
│                   [-0.1918,  0.9390,  0.5160]])
└── 'x' --> <FastTreeValue 0x7fc4b4af3160>
    └── 'c' --> tensor([[-1.1935, -0.9629, -0.5751,  1.0858],
                        [ 1.5296,  0.4821,  0.2280,  0.8139],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 279 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0973,  0.9077,  0.9042],
                [-0.1918,  0.9390,  0.5160]],
       
               [[ 0.0973,  0.9077,  0.9042],
                [-0.1918,  0.9390,  0.5160]],
       
               [[ 0.0973,  0.9077,  0.9042],
                [-0.1918,  0.9390,  0.5160]],
       
               [[ 0.0973,  0.9077,  0.9042],
                [-0.1918,  0.9390,  0.5160]],
       
               [[ 0.0973,  0.9077,  0.9042],
                [-0.1918,  0.9390,  0.5160]],
       
               [[ 0.0973,  0.9077,  0.9042],
                [-0.1918,  0.9390,  0.5160]],
       
               [[ 0.0973,  0.9077,  0.9042],
                [-0.1918,  0.9390,  0.5160]],
       
               [[ 0.0973,  0.9077,  0.9042],
                [-0.1918,  0.9390,  0.5160]]]),
    x: Batch(
           c: tensor([[[-1.1935, -0.9629, -0.5751,  1.0858],
                       [ 1.5296,  0.4821,  0.2280,  0.8139],
                       [ 0.1560, -0.4275, -0.7215, -0.3529]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160],
               [ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160],
               [ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160],
               [ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160],
               [ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160],
               [ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160],
               [ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160],
               [ 0.0973,  0.9077,  0.9042],
               [-0.1918,  0.9390,  0.5160]]),
    x: Batch(
           c: tensor([[-1.1935, -0.9629, -0.5751,  1.0858],
                      [ 1.5296,  0.4821,  0.2280,  0.8139],
                      [ 0.1560, -0.4275, -0.7215, -0.3529],
                      [-1.1935, -0.9629, -0.5751,  1.0858],
                      [ 1.5296,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 325 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 4.59 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
